In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
train_data=pd.read_csv('Train_Dataset.csv')
test_data=pd.read_csv('Test_Dataset.csv')

In [3]:
train_data.shape

(7810, 22)

In [4]:
train_data.duplicated().sum()

2629

In [5]:
train_data=train_data.drop_duplicates()

In [6]:
train_data.duplicated().sum()

0

In [7]:
train_data.shape

(5181, 22)

In [8]:
test_data.shape

(2630, 21)

In [9]:
test_data.duplicated().sum()

0

In [10]:
train_data.head(2)

,EmployeeID,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,...,JobSatisfaction,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome
0,5110001.0,0.0,35.0,Rarely,Analytics,5.0,CA,Male,69.0,1.0,...,1.0,1.0,1.0,1.0,20.0,7.0,2.0,NaN,M,18932.0
1,5110002.0,1.0,32.0,Yes,Sales,5.0,Statistics,Female,62.0,4.0,...,2.0,0.0,8.0,0.0,20.0,4.0,1.0,NaN,Single,18785.0


In [11]:
test_data.head(2)

,EmployeeID,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,JobSatisfaction,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome
0,6110001,18.0,No,NaN,9.0,CA,Male,80.0,3,2,...,3,1,1,0,17.0,0.0,0.0,0.0,Single,16904.0
1,6110002,20.0,Rarely,Analytics,28.0,Statistics,Female,59.0,1,3,...,1,1,2,1,18.0,2.0,1.0,NaN,Single,18994.0


In [12]:
train_data['data']='train'
test_data['data']='test'

In [13]:
total=pd.concat([train_data,test_data],axis=0)
total.head()

,EmployeeID,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
0,5110001.0,0.0,35.0,Rarely,Analytics,5.0,CA,Male,69.0,1.0,...,1.0,1.0,1.0,20.0,7.0,2.0,NaN,M,18932.0,train
1,5110002.0,1.0,32.0,Yes,Sales,5.0,Statistics,Female,62.0,4.0,...,0.0,8.0,0.0,20.0,4.0,1.0,NaN,Single,18785.0,train
2,5110003.0,0.0,31.0,Rarely,Analytics,5.0,Statistics,F,45.0,5.0,...,1.0,3.0,0.0,26.0,12.0,1.0,3.0,Single,22091.0,train
3,5110004.0,0.0,34.0,Yes,Sales,10.0,Statistics,Female,32.0,3.0,...,1.0,1.0,0.0,23.0,5.0,1.0,3.0,Divorsed,20302.0,train
4,5110005.0,0.0,37.0,No,Analytics,27.0,Statistics,Female,49.0,3.0,...,1.0,8.0,0.0,21.0,12.0,1.0,9.0,Divorsed,21674.0,train


In [14]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7811 entries, 0 to 2629
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EmployeeID          7810 non-null   float64
 1   Attrition           5180 non-null   float64
 2   Age                 7352 non-null   float64
 3   TravelProfile       7810 non-null   object 
 4   Department          7628 non-null   object 
 5   HomeToWork          7429 non-null   float64
 6   EducationField      7810 non-null   object 
 7   Gender              7734 non-null   object 
 8   HourlnWeek          7387 non-null   float64
 9   Involvement         7810 non-null   float64
 10  WorkLifeBalance     7810 non-null   float64
 11  Designation         7742 non-null   object 
 12  JobSatisfaction     7810 non-null   float64
 13  ESOPs               7810 non-null   float64
 14  NumCompaniesWorked  7810 non-null   float64
 15  OverTime            7810 non-null   float64
 16  Salary

In [15]:
total=total.drop(['EmployeeID'],axis=1)

In [16]:
total.describe(include='all')

,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
count,5180.000000,7352.000000,7810,7628,7429.000000,7810,7734,7387.000000,7810.000000,7810.000000,...,7810.000000,7810.000000,7810.000000,7547.000000,7501.000000,7683.000000,7365.000000,7810,7684.000000,7811
unique,NaN,NaN,3,3,NaN,6,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,2
top,NaN,NaN,Rarely,Analytics,NaN,Statistics,Male,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single,NaN,train
freq,NaN,NaN,5489,4894,NaN,3169,4668,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2709,NaN,5181
mean,0.278958,37.215860,NaN,NaN,11.215507,NaN,NaN,57.940436,3.230986,3.031754,...,0.495391,3.179385,0.319078,20.645157,11.286095,2.589093,4.426477,NaN,21736.822098,NaN
std,0.448530,9.286258,NaN,NaN,8.590705,NaN,NaN,13.076675,0.876355,1.412770,...,0.500011,2.625605,0.466149,3.712639,7.910095,3.257934,3.606972,NaN,4881.903674,NaN
min,0.000000,18.000000,NaN,NaN,1.000000,NaN,NaN,10.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,NaN,1000.000000,NaN
25%,0.000000,30.000000,NaN,NaN,5.000000,NaN,NaN,49.000000,3.000000,2.000000,...,0.000000,1.000000,0.000000,18.000000,6.000000,1.000000,2.000000,NaN,18377.000000,NaN
50%,0.000000,36.000000,NaN,NaN,9.000000,NaN,NaN,59.000000,3.000000,3.000000,...,0.000000,2.000000,0.000000,20.000000,10.000000,1.000000,3.000000,NaN,20229.000000,NaN
75%,1.000000,43.000000,NaN,NaN,16.000000,NaN,NaN,67.000000,4.000000,4.000000,...,1.000000,5.000000,1.000000,23.000000,15.000000,3.000000,7.000000,NaN,23465.750000,NaN


In [17]:
def missing_values(col):
    total[col]=total[col].fillna(total[col].median())

In [18]:
miss_col=['HomeToWork','HourlnWeek','NumCompaniesWorked','SalaryHikelastYear','WorkExperience','LastPromotion',
         'MonthlyIncome','Involvement','WorkLifeBalance','JobSatisfaction','ESOPs','OverTime','Age','CurrentProfile']
miss_col

['HomeToWork',
 'HourlnWeek',
 'NumCompaniesWorked',
 'SalaryHikelastYear',
 'WorkExperience',
 'LastPromotion',
 'MonthlyIncome',
 'Involvement',
 'WorkLifeBalance',
 'JobSatisfaction',
 'ESOPs',
 'OverTime',
 'Age',
 'CurrentProfile']

In [19]:
for i in miss_col[0:]:
    missing_values(i)

In [20]:
total['Gender'].value_counts()

Male      4668
Female    2020
F         1046
Name: Gender, dtype: int64

In [21]:
total['Gender']=np.where(total['Gender']=='F','Female',total['Gender'])

In [22]:
total['Gender'].value_counts()

Male      4668
Female    3066
Name: Gender, dtype: int64

In [23]:
total['MaritalStatus'].value_counts()

Single      2709
Married     2455
Divorsed    1493
M           1153
Name: MaritalStatus, dtype: int64

In [24]:
total['MaritalStatus']=np.where(total['MaritalStatus']=='M','Married',total['MaritalStatus'])

In [25]:
total['MaritalStatus'].value_counts()

Married     3608
Single      2709
Divorsed    1493
Name: MaritalStatus, dtype: int64

In [26]:
total['Department']=total['Department'].fillna('Analytics')
total['TravelProfile']=total['TravelProfile'].fillna('Rarely')
total['EducationField']=total['EducationField'].fillna('Statistics')
total['Gender']=total['Gender'].fillna('Male')
total['Designation']=total['Designation'].fillna('Executive')
total['MaritalStatus']=total['MaritalStatus'].fillna('Married')

In [27]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7811 entries, 0 to 2629
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Attrition           5180 non-null   float64
 1   Age                 7811 non-null   float64
 2   TravelProfile       7811 non-null   object 
 3   Department          7811 non-null   object 
 4   HomeToWork          7811 non-null   float64
 5   EducationField      7811 non-null   object 
 6   Gender              7811 non-null   object 
 7   HourlnWeek          7811 non-null   float64
 8   Involvement         7811 non-null   float64
 9   WorkLifeBalance     7811 non-null   float64
 10  Designation         7811 non-null   object 
 11  JobSatisfaction     7811 non-null   float64
 12  ESOPs               7811 non-null   float64
 13  NumCompaniesWorked  7811 non-null   float64
 14  OverTime            7811 non-null   float64
 15  SalaryHikelastYear  7811 non-null   float64
 16  WorkEx

In [28]:
total['Involvement']=total['Involvement'].astype('str')
total['WorkLifeBalance']=total['WorkLifeBalance'].astype('str')
total['JobSatisfaction']=total['JobSatisfaction'].astype('str')
total['ESOPs']=total['ESOPs'].astype('str')
total['OverTime']=total['OverTime'].astype('str')

In [29]:
total.describe(include='all')

,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
count,5180.000000,7811.000000,7811,7811,7811.000000,7811,7811,7811.000000,7811,7811,...,7811,7811.000000,7811,7811.000000,7811.000000,7811.000000,7811.000000,7811,7811.000000,7811
unique,NaN,NaN,3,3,NaN,6,2,NaN,5,5,...,2,NaN,2,NaN,NaN,NaN,NaN,3,NaN,2
top,NaN,NaN,Rarely,Analytics,NaN,Statistics,Male,NaN,3.0,4.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,Married,NaN,train
freq,NaN,NaN,5490,5077,NaN,3170,4745,NaN,4541,1635,...,3942,NaN,5319,NaN,NaN,NaN,NaN,3609,NaN,5181
mean,0.278958,37.144412,NaN,NaN,11.107157,NaN,NaN,57.997952,NaN,NaN,...,NaN,3.179234,NaN,20.623352,11.235053,2.563052,4.345026,NaN,21712.306235,NaN
std,0.448530,9.013783,NaN,NaN,8.391595,NaN,NaN,12.719026,NaN,NaN,...,NaN,2.625470,NaN,3.651213,7.755585,3.237419,3.518074,NaN,4845.802273,NaN
min,0.000000,18.000000,NaN,NaN,1.000000,NaN,NaN,10.000000,NaN,NaN,...,NaN,0.000000,NaN,16.000000,0.000000,0.000000,0.000000,NaN,1000.000000,NaN
25%,0.000000,31.000000,NaN,NaN,5.000000,NaN,NaN,50.000000,NaN,NaN,...,NaN,1.000000,NaN,18.000000,6.000000,1.000000,2.000000,NaN,18398.000000,NaN
50%,0.000000,36.000000,NaN,NaN,9.000000,NaN,NaN,59.000000,NaN,NaN,...,NaN,2.000000,NaN,20.000000,10.000000,1.000000,3.000000,NaN,20229.000000,NaN
75%,1.000000,43.000000,NaN,NaN,15.000000,NaN,NaN,67.000000,NaN,NaN,...,NaN,5.000000,NaN,23.000000,15.000000,3.000000,7.000000,NaN,23380.500000,NaN


In [30]:
label_encoder=['Gender','ESOPs','OverTime']
label_encoder

['Gender', 'ESOPs', 'OverTime']

In [31]:
total['Gender']=total['Gender'].astype('category')
total['Gender']=total['Gender'].cat.codes
total['ESOPs']=total['ESOPs'].astype('category')
total['ESOPs']=total['ESOPs'].cat.codes
total['OverTime']=total['OverTime'].astype('category')
total['OverTime']=total['OverTime'].cat.codes

In [32]:
total.describe(include='all')

,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
count,5180.000000,7811.000000,7811,7811,7811.000000,7811,7811.000000,7811.000000,7811,7811,...,7811.000000,7811.000000,7811.000000,7811.000000,7811.000000,7811.000000,7811.000000,7811,7811.000000,7811
unique,NaN,NaN,3,3,NaN,6,NaN,NaN,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,2
top,NaN,NaN,Rarely,Analytics,NaN,Statistics,NaN,NaN,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Married,NaN,train
freq,NaN,NaN,5490,5077,NaN,3170,NaN,NaN,4541,1635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3609,NaN,5181
mean,0.278958,37.144412,NaN,NaN,11.107157,NaN,0.607477,57.997952,NaN,NaN,...,0.495327,3.179234,0.319037,20.623352,11.235053,2.563052,4.345026,NaN,21712.306235,NaN
std,0.448530,9.013783,NaN,NaN,8.391595,NaN,0.488343,12.719026,NaN,NaN,...,0.500010,2.625470,0.466133,3.651213,7.755585,3.237419,3.518074,NaN,4845.802273,NaN
min,0.000000,18.000000,NaN,NaN,1.000000,NaN,0.000000,10.000000,NaN,NaN,...,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,NaN,1000.000000,NaN
25%,0.000000,31.000000,NaN,NaN,5.000000,NaN,0.000000,50.000000,NaN,NaN,...,0.000000,1.000000,0.000000,18.000000,6.000000,1.000000,2.000000,NaN,18398.000000,NaN
50%,0.000000,36.000000,NaN,NaN,9.000000,NaN,1.000000,59.000000,NaN,NaN,...,0.000000,2.000000,0.000000,20.000000,10.000000,1.000000,3.000000,NaN,20229.000000,NaN
75%,1.000000,43.000000,NaN,NaN,15.000000,NaN,1.000000,67.000000,NaN,NaN,...,1.000000,5.000000,1.000000,23.000000,15.000000,3.000000,7.000000,NaN,23380.500000,NaN


In [33]:
total['TravelProfile']=total['TravelProfile'].astype('category')
total['TravelProfile']=total['TravelProfile'].cat.codes
total['Department']=total['Department'].astype('category')
total['Department']=total['Department'].cat.codes
total['EducationField']=total['EducationField'].astype('category')
total['EducationField']=total['EducationField'].cat.codes
total['Involvement']=total['Involvement'].astype('category')
total['Involvement']=total['Involvement'].cat.codes
total['WorkLifeBalance']=total['WorkLifeBalance'].astype('category')
total['WorkLifeBalance']=total['WorkLifeBalance'].cat.codes
total['MaritalStatus']=total['MaritalStatus'].astype('category')
total['MaritalStatus']=total['MaritalStatus'].cat.codes
total['Designation']=total['Designation'].astype('category')
total['Designation']=total['Designation'].cat.codes

In [34]:
total.head()

,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
0,0.0,35.0,1,0,5.0,0,1,69.0,0,0,...,1,1.0,1,20.0,7.0,2.0,3.0,1,18932.0,train
1,1.0,32.0,2,2,5.0,5,0,62.0,3,2,...,0,8.0,0,20.0,4.0,1.0,3.0,2,18785.0,train
2,0.0,31.0,1,0,5.0,5,0,45.0,4,2,...,1,3.0,0,26.0,12.0,1.0,3.0,2,22091.0,train
3,0.0,34.0,2,2,10.0,5,0,32.0,2,1,...,1,1.0,0,23.0,5.0,1.0,3.0,0,20302.0,train
4,0.0,37.0,0,0,27.0,5,0,49.0,2,3,...,1,8.0,0,21.0,12.0,1.0,9.0,0,21674.0,train


In [35]:
train=total[total['data']=='train']
test=total[total['data']=='test']

In [36]:
train['Attrition']=train['Attrition'].astype('category')
train['Attrition']=train['Attrition'].cat.codes

In [37]:
train.head()

,Attrition,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,data
0,0,35.0,1,0,5.0,0,1,69.0,0,0,...,1,1.0,1,20.0,7.0,2.0,3.0,1,18932.0,train
1,1,32.0,2,2,5.0,5,0,62.0,3,2,...,0,8.0,0,20.0,4.0,1.0,3.0,2,18785.0,train
2,0,31.0,1,0,5.0,5,0,45.0,4,2,...,1,3.0,0,26.0,12.0,1.0,3.0,2,22091.0,train
3,0,34.0,2,2,10.0,5,0,32.0,2,1,...,1,1.0,0,23.0,5.0,1.0,3.0,0,20302.0,train
4,0,37.0,0,0,27.0,5,0,49.0,2,3,...,1,8.0,0,21.0,12.0,1.0,9.0,0,21674.0,train


In [38]:
train['Attrition'].value_counts()

 0    3735
 1    1445
-1       1
Name: Attrition, dtype: int64

In [39]:
train['Attrition']=np.where(train['Attrition']==-1,0,train['Attrition'])

In [40]:
train['Attrition'].value_counts()

0    3736
1    1445
Name: Attrition, dtype: int64

In [41]:
train=train.drop(columns=['data'],axis=1)
test=test.drop(columns=['data','Attrition'],axis=1)

In [42]:
x=train.iloc[:,1:]
y=train.iloc[:,0]

In [43]:
y.value_counts()

0    3736
1    1445
Name: Attrition, dtype: int64

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=42,stratify=y)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(4144, 20) (1037, 20) (4144,) (1037,)


In [45]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [46]:
rf=RandomForestClassifier(n_estimators=300,criterion='gini')
rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=300)

In [47]:
train_pred=rf.predict(x_train)
test_pred=rf.predict(x_test)

In [48]:
print(classification_report(y_train,train_pred))
print(classification_report(y_test,test_pred))
print()
print(confusion_matrix(y_train,train_pred))
print(confusion_matrix(y_test,test_pred))
print()
print(accuracy_score(y_train,train_pred))
print(accuracy_score(y_test,test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2988
           1       1.00      1.00      1.00      1156

    accuracy                           1.00      4144
   macro avg       1.00      1.00      1.00      4144
weighted avg       1.00      1.00      1.00      4144

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       748
           1       0.99      0.95      0.97       289

    accuracy                           0.98      1037
   macro avg       0.99      0.97      0.98      1037
weighted avg       0.98      0.98      0.98      1037


[[2988    0]
 [   0 1156]]
[[745   3]
 [ 14 275]]

1.0
0.9836065573770492


In [49]:
cvs=cross_val_score(rf,x_train,y_train,cv=10)
print('10 accuracy is: ',cvs)
print('Avg accuracy is: ',cvs.mean())
print('Max accuracy is: ',cvs.max())

10 accuracy is:  [0.99759036 0.97590361 0.97590361 0.97831325 0.97826087 0.97584541
 0.97342995 0.98067633 0.97101449 0.98067633]
Avg accuracy is:  0.9787614225016006
Max accuracy is:  0.9975903614457832


In [51]:
test1=scaler.transform(test)
pd.DataFrame(test1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-2.124964,-2.074479,-0.704039,-0.267333,-1.204090,0.799782,1.713108,-0.270986,-0.754935,-0.748929,-0.086422,0.990162,-0.834645,-0.701870,-0.987579,-1.442745,-0.801358,-1.262884,1.161991,-0.967866
1,-1.905315,-0.199606,-0.704039,1.929807,1.063061,-1.250341,0.083642,-2.533508,-0.048861,-0.748929,-1.581743,0.990162,-0.459154,1.424765,-0.714319,-1.190842,-0.496090,-0.407665,1.161991,-0.555043
2,1.389430,-0.199606,-0.704039,0.889057,-1.204090,-1.250341,1.402734,-0.270986,-0.048861,-0.748929,1.408900,-1.009935,0.667316,1.424765,0.378719,0.824385,0.114446,-0.407665,-0.243730,-0.635435
3,-0.587417,-0.199606,1.488528,1.351612,1.063061,-1.250341,1.169953,1.991535,-0.754935,0.302129,-0.086422,-1.009935,0.291826,1.424765,-0.987579,-0.812987,0.114446,-0.407665,-0.243730,-0.245921
4,0.181357,-0.199606,-0.704039,-0.498610,-1.204090,0.799782,-1.235450,0.860274,-1.461009,1.353188,1.408900,0.990162,0.291826,-0.701870,-0.167800,-0.309180,-0.496090,0.732627,-0.243730,0.629500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,0.950131,-0.199606,1.488528,0.079584,0.156201,0.799782,1.402734,-0.270986,1.363286,1.353188,1.408900,0.990162,0.667316,1.424765,-0.714319,-0.183228,-0.496090,-0.407665,-0.243730,0.979115
2626,-0.916891,-0.199606,-0.704039,1.235973,-1.204090,0.799782,1.713108,0.860274,0.657212,-0.748929,1.408900,-1.009935,-0.459154,-0.701870,-0.714319,-0.561083,0.419715,-0.122592,-1.649451,-0.515144
2627,0.730481,-0.199606,-0.704039,-0.382971,-1.204090,-1.250341,-1.235450,0.860274,-1.461009,1.353188,-1.581743,0.990162,-0.083664,-0.701870,-0.441060,1.580094,0.724983,3.583356,-0.243730,0.680263
2628,-0.148117,-0.199606,-0.704039,-0.036055,1.063061,-1.250341,0.083642,0.860274,-0.048861,-0.748929,0.661239,-1.009935,-0.459154,-0.701870,-0.714319,-1.190842,-0.496090,-0.977811,1.161991,-0.897350


In [52]:
final=rf.predict(test1)
final

array([1, 0, 0, ..., 0, 1, 0], dtype=int8)

In [53]:
print(len(final))
print(len(test1))

2630
2630


In [55]:
final_output=pd.concat([test,pd.DataFrame(final)],axis=1)
final_output.head()

,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,Designation,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,0
0,18.0,0,0,9.0,0,1,80.0,2,1,1,...,1,1.0,0,17.0,0.0,0.0,0.0,2,16904.0,1
1,20.0,1,0,28.0,5,0,59.0,0,2,1,...,1,2.0,1,18.0,2.0,1.0,3.0,2,18994.0,0
2,50.0,1,0,19.0,0,0,76.0,2,2,1,...,0,5.0,1,22.0,18.0,3.0,3.0,1,18587.0,0
3,32.0,1,2,23.0,5,0,73.0,4,1,2,...,0,4.0,1,17.0,5.0,3.0,3.0,1,20559.0,1
4,39.0,1,0,7.0,0,1,42.0,3,0,3,...,1,4.0,0,20.0,9.0,1.0,7.0,1,24991.0,0


In [57]:
final_output.to_csv('Final Report.csv')